In [4]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q http://archive.apache.org/dist/spark/spark-3.1.1/spark-3.1.1-bin-hadoop3.2.tgz
!tar xf spark-3.1.1-bin-hadoop3.2.tgz
!pip install -q findspark
import os

^C

gzip: stdin: unexpected end of file
tar: Unexpected EOF in archive
tar: Unexpected EOF in archive
tar: Error is not recoverable: exiting now


In [5]:
!apt-get update
!apt-get install openjdk-11-jdk -y
!pip install pyspark

Get:1 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease [1,581 B]
Get:2 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,632 B]
Hit:3 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:4 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Get:5 https://r2u.stat.illinois.edu/ubuntu jammy InRelease [6,555 B]
Get:6 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Get:7 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  Packages [1,702 kB]
Get:8 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease [18.1 kB]
Get:9 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [127 kB]
Hit:10 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:11 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Get:12 https://r2u.stat.illinois.edu/ubuntu jammy/main all Packages [8,968 kB]
Get:13 http://security.ubuntu.com/ubuntu

In [6]:

os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = "/usr/local/lib/python3.11/dist-packages/pyspark"

In [7]:
!pip install pyngrok
from pyngrok import ngrok

In [20]:
#bring in Pyspark functions into your session
from pyspark.sql.functions import *
from pyspark.sql.functions import col

In [9]:
#Start our Pyspark session
from pyspark.sql import SparkSession
spark = SparkSession.builder.master("local[*]").getOrCreate()


•	Reading and Writing Data

In [18]:


# Create simple dataframe
df = spark.createDataFrame([(1, "foo"), (2, "bar"), (1, "foo")], ["id", "value"])
df.show()

+---+-----+
| id|value|
+---+-----+
|  1|  foo|
|  2|  bar|
|  1|  foo|
+---+-----+



In [11]:
mydata = spark.read.format("csv").option("header","true").load("original.csv")

In [12]:
# prompt: import json file

import json
df2 = spark.read.json('people.json')
df2.show()




+----+-------+
| age|   name|
+----+-------+
|NULL|Michael|
|  30|   Andy|
|  19| Justin|
+----+-------+



Data Checking/cleaning & *Validation*

In [13]:
mydata.printSchema()

root
 |-- id: string (nullable = true)
 |-- first_name: string (nullable = true)
 |-- last_name: string (nullable = true)
 |-- gender: string (nullable = true)
 |-- City: string (nullable = true)
 |-- JobTitle: string (nullable = true)
 |-- Salary: string (nullable = true)
 |-- Latitude: string (nullable = true)
 |-- Longitude: string (nullable = true)



In [14]:
#Exploratory Functions


df.describe()



DataFrame[summary: string, id: string, value: string]

In [15]:
df.summary()


DataFrame[summary: string, id: string, value: string]

In [16]:
mydata.columns


['id',
 'first_name',
 'last_name',
 'gender',
 'City',
 'JobTitle',
 'Salary',
 'Latitude',
 'Longitude']

In [17]:
df2.dtypes

[('age', 'bigint'), ('name', 'string')]

In [21]:
#Null Checking & Missing Value Analysis
mydata.select([
    sum(when(col(c).isNull(), 1).otherwise(0)).alias(c)
    for c in mydata.columns
]).show()


mydata.filter(mydata.col.isNull())
#Drop Rows with Any Null
mydata.na.drop()
mydata.na.fill(0)




+---+----------+---------+------+----+--------+------+--------+---------+
| id|first_name|last_name|gender|City|JobTitle|Salary|Latitude|Longitude|
+---+----------+---------+------+----+--------+------+--------+---------+
|  0|         0|        0|     0|   1|       2|     0|       1|        0|
+---+----------+---------+------+----+--------+------+--------+---------+



In [ ]:
#Replace Nulls Only in Selected Columns

df_filled = df2.fillna(0, subset=['age'])
df_filled .show()

+---+-------+
|age|   name|
+---+-------+
|  0|Michael|
| 30|   Andy|
| 19| Justin|
+---+-------+



In [26]:
# Duplicate Detection


df.groupBy(df.columns) \
  .count() \
  .filter("count > 1") \
  .show()




df.dropDuplicates().show()

df_drop= df.dropDuplicates()
df_drop.show()

+---+-----+
| id|value|
+---+-----+
|  1|  foo|
|  2|  bar|
+---+-----+



Data Mining & Feature Engineering

In [ ]:
#Filtering and Conditions
filter()



chained conditions using &, |, ~

In [ ]:

from pyspark.sql.functions import col


In [27]:
data = [
    (1, "Alice", 29),
    (2, "Bob", 35),
    (3, "Eve", 25),
]

df = spark.createDataFrame(data, ["id", "name", "age"])

# Simple condition: age > 30 OR name == "Eve"
filtered_df = df.filter((col("age") > 30) | (col("name") == "Eve"))



In [28]:
filtered_df.show()

+---+----+---+
| id|name|age|
+---+----+---+
|  2| Bob| 35|
|  3| Eve| 25|
+---+----+---+



In PySpark, you can use chained conditions with the bitwise operators:
•	& for AND
•	| for OR
•	~ for NOT
These are used inside filter() or where() clauses, and each condition must be enclosed in parentheses to avoid operator precedence issues.


In [29]:

from pyspark.sql.functions import col



In [40]:
# Sample data
data = [
    (1, "Alice", 29),
    (2, "Bob", 35),
    (3, "Charlie", 30),
    (4, "Diana", 40),
    (5, "Eve", 25)
]

# Create DataFrame
df = spark.createDataFrame(data, ["id", "name", "age"])
df.show()


+---+-------+---+
| id|   name|age|
+---+-------+---+
|  1|  Alice| 29|
|  2|    Bob| 35|
|  3|Charlie| 30|
|  4|  Diana| 40|
|  5|    Eve| 25|
+---+-------+---+



In [43]:
# Chained filter with &, |, ~
# Show me those who are over 30 years of age and excluded anyone named Diana,
filtered_df = df.filter(
    ((col("age") > 30) & (col("name") != "Diana"))
)

# Show result
filtered_df.show()

filtered_df = df.filter(
    ((col("age") > 30) & (col("name") != "Diana")) | (~(col("age") < 30))
)

# Show result
filtered_df.show()

+---+----+---+
| id|name|age|
+---+----+---+
|  2| Bob| 35|
+---+----+---+

